# Importing the Required Libraries 

In [42]:
# loading the reqed libraries for scraping from Twitter and Reddit 
import tweepy 
import hvplot.pandas
import praw
import os  
from dotenv import load_dotenv
import torch 
import pandas as pd
import praw
from matplotlib import pyplot as plt 
%matplotlib inline 





In [2]:
# Loading the env varibles for the twitter and Reddit 
load_dotenv()
twit_api= os.getenv("TWIT_API")
twit_secret = os.getenv("TWIT_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# Grabbing the Twitter Data

In [3]:

# authentication
auth = tweepy.OAuthHandler(twit_api, twit_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


# tweets = tweepy.Cursor(api.user_timeline, screen_name=user, count=200, tweet_mode='extended').items(limit)

# search tweets
keywords = 'BTC'
limit=300

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=10, tweet_mode='extended').items(limit)

# tweets = api.user_timeline(screen_name=user, count=limit, tweet_mode='extended')

# create DataFrame
columns = ['User', 'Tweet']
data = []

for tweet in tweets:
    data.append([tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)

print(df)
df.to_csv('NVDA.csv')

                User                                              Tweet
0     taylorjulien35  RT @convexical: Converting $5,000,000 of #btc ...
1          Mr_chispa  @mis2centavos Para cositorto esa bandeja esta ...
2      aidenkang_btc  RT @ant_stock_park: ㅋㅋㅋㅋㅋㅋ 오픈씨에 박아버렸네 ㅋㅋㅋㅋㅋㅋ h...
3    CryptoRomeroMan  🇺🇸🇬🇧 Welcome to the official Twitter account o...
4           kedai025  RT @Heather99924565: Wow #2112Run is doing a l...
..               ...                                                ...
295        coder_487  RT @NewsQnt: Everyone knows of $BTC and $ETH …...
296       robostopia  💰 Market Cap: $2,017,893,783,996\n👊 BTC Domina...
297         lemunche  RT @rick_bitcoin: My Brother #XVG $XVG 🚀🚀🚀 #BT...
298     thebitofcoin  He just hates #btc so much, imagine living whi...
299   criptohispania  Rojan Crypto news\n▶ https://t.co/gHuToyMoRx\n...

[300 rows x 2 columns]


In [4]:
import praw

In [5]:
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_SEC"),
    password=os.getenv("REDDIT_PASS"),
    user_agent="stock_dashboard by u/CryptoFoDayz",
    username=os.getenv("REDDIT_USER"),
)
reddit.read_only = True


In [6]:
for subreddit in reddit.subreddits.default(limit=10):
    print(subreddit)

gadgets
sports
gaming
pics
worldnews
videos
AskReddit
aww
Music
funny


# Perform Sentiment Analysis 

In [7]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# create tokenizer variable to chose the model 
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")



In [12]:
# encode the list of tweet to create the sentiment score from a function
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [13]:
df['sentiment'] = df['Tweet'].apply(lambda x: sentiment_score(x[:512]))


In [48]:
df_grouped= df.groupby(by = 'sentiment').count()
df_grouped.reset_index()
df_grouped.drop(columns ='User' , inplace = True)

In [49]:
df_grouped.hvplot(kind = 'bar')

:Bars   [sentiment]   (Tweet)

In [50]:
df_grouped.head()

,Tweet
sentiment,
1,162
2,3
3,13
4,8
5,114
